# Deep Neural Networks

## Load Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [3]:
met0_tr = pd.read_pickle('kaggle/prepo_train_met0_bIDbyM_means.pkl')
met0_vl = pd.read_pickle('kaggle/prepo_val_met0__bIDbyM_means.pkl')

In [4]:
xs = [x for x in list(met0_tr) if x not in ['timestamp', 'meter_reading']]
y = 'meter_reading'

In [5]:
met0_tr_x = met0_tr[xs]
met0_tr_y = met0_tr[y]
met0_vl_x = met0_vl[xs]
met0_vl_y = met0_vl[y]

In [6]:
del met0_tr, met0_vl

## Build Network

In [7]:
import tensorflow as tf
from keras import layers

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [8]:
import keras

In [44]:
tf.reset_default_graph()

In [9]:
def build_model(no_of_feat):
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[no_of_feat]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='relu')
    ])
    model.summary()
    model.compile(loss=root_mean_squared_logarithmic_error, optimizer='rmsprop', metrics=['mse'])
    return model

In [10]:
import keras.backend as K

In [11]:
def root_mean_squared_logarithmic_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))))

In [12]:
model = build_model(len(list(met0_tr_x)))




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                3712      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 7,937
Trainable params: 7,937
Non-trainable params: 0
_________________________________________________________________



In [14]:
early_stopper = keras.callbacks.EarlyStopping(patience=20, verbose=1)
check_pointer = keras.callbacks.ModelCheckpoint('models/DNN_MT0_261019T1500.json', verbose=1, save_best_only=True)

In [15]:
model.fit(met0_tr_x, met0_tr_y, epochs=10, verbose=1, batch_size=256,
          validation_data=[met0_vl_x, met0_vl_y], callbacks=[early_stopper, check_pointer])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10661397 samples, validate on 1399513 samples
Epoch 1/10
10661397/10661397 [==============================] - 150s 14us/step - loss: 4.3279 - mean_squared_error: 204065.9700 - val_loss: 4.3002 - val_mean_squared_error: 174434.1227

Epoch 00001: val_loss improved from inf to 4.30023, saving model to models/DNN_MT0_261019T1500.json
Epoch 2/10
10661397/10661397 [==============================] - 145s 14us/step - loss: 4.3278 - mean_squared_error: 174187.5428 - val_loss: 4.3002 - val_mean_squared_error: 174434.1227

Epoch 00002: val_loss did not improve from 4.30023
Epoch 3/10
 3271936/10661397 [========>.....................] - ETA: 1:35 - loss: 4.3263 - mean_squared_error: 175501.9525

KeyboardInterrupt: 

In [23]:
from tsm.eval_metrics import root_mean_squared_log_error

In [18]:
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

In [19]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    model = load_model('models/DNN_MT0_261019T1415.json')

TypeError: Unexpected keyword argument passed to optimizer: momentum

In [38]:
#dnn_tr_hat = (model.predict(met0_tr_x))
#print('Training NRMLSE:', root_mean_squared_log_error(dnn_tr_hat, met0_tr_y))

dnn_vl_hat = (model.predict(met0_vl_x))
vl_nrmlse = root_mean_squared_log_error(dnn_vl_hat, met0_vl_y)
print('Validation NRMLSE:', vl_nrmlse)

/home/Ric/TSM/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


MemoryError: Unable to allocate array with shape (1399513, 1399513) and data type float32